# Olist : maintenance

In [39]:
#librairies import

import pandas as pd
import numpy as np
import scipy as scp
import matplotlib.pyplot as plt
import seaborn as sns

import joblib

from sklearn.preprocessing import StandardScaler

from sklearn.cluster import KMeans

from scipy.stats import kstest, norm, expon, uniform

We will first use the ks test to evaluate if our data follows a certain logic

In [40]:
data = pd.read_csv('data/data_with_clustering.csv')
data.head()

,last_purchase,days_since_last_purchase,total_price_items,km_cluster,total_freight_value,total_orders,km_enriched_cluster,DBSCAN_cluster
0,1.534792e+09,2129,729.62,0,172.42,17,4,-1
1,1.519757e+09,2303,1000.85,3,171.82,9,4,-1
2,1.530147e+09,2182,664.20,0,94.63,7,4,-1
3,1.527853e+09,2209,806.61,0,316.11,7,4,-1
4,1.518615e+09,2316,855.21,3,165.91,7,4,-1


In [33]:
#Model import

joblib_file = "kmeans_model.pkl"
kmeans_original = joblib.load(joblib_file)

### ARI analysis

We will use the KS test to establish which statistical law is the fittest to simulate data evolution, then use the ARI to see when changes are to important, therefore calling for a retraining of the model

In [41]:
data = data.drop(columns = ['km_enriched_cluster', 'DBSCAN_cluster', 'km_cluster'])
data.head()

,last_purchase,days_since_last_purchase,total_price_items,total_freight_value,total_orders
0,1.534792e+09,2129,729.62,172.42,17
1,1.519757e+09,2303,1000.85,171.82,9
2,1.530147e+09,2182,664.20,94.63,7
3,1.527853e+09,2209,806.61,316.11,7
4,1.518615e+09,2316,855.21,165.91,7


In [42]:
# Iterate over each column (feature) in the dataframe

for col in data.columns:
    # Extract the numpy array for the current feature
    feature_array = data[col].values

    # Perform the Kolmogorov-Smirnov test
    D, p = stats.kstest(feature_array, 'norm')  # Test against a normal distribution

    # Print the results
    print(f"Feature: {col}")
    print(f"D-statistic: {D:.4f}, p-value: {p:.4f}")
    print("---")

Feature: last_purchase
D-statistic: 1.0000, p-value: 0.0000
---
Feature: days_since_last_purchase
D-statistic: 1.0000, p-value: 0.0000
---
Feature: total_price_items
D-statistic: 0.9999, p-value: 0.0000
---
Feature: total_freight_value
D-statistic: 0.9965, p-value: 0.0000
---
Feature: total_orders
D-statistic: 0.8413, p-value: 0.0000
---


In [43]:
# Iterate over each column (feature) in the dataframe

for col in data.columns:
    # Extract the numpy array for the current feature
    feature_array = data[col].values

    # Perform the Kolmogorov-Smirnov test
    D, p = stats.kstest(feature_array, 'expon')  # Test against a normal distribution

    # Print the results
    print(f"Feature: {col}")
    print(f"D-statistic: {D:.4f}, p-value: {p:.4f}")
    print("---")

Feature: last_purchase
D-statistic: 1.0000, p-value: 0.0000
---
Feature: days_since_last_purchase
D-statistic: 1.0000, p-value: 0.0000
---
Feature: total_price_items
D-statistic: 0.9978, p-value: 0.0000
---
Feature: total_freight_value
D-statistic: 0.9957, p-value: 0.0000
---
Feature: total_orders
D-statistic: 0.6321, p-value: 0.0000
---


In [ ]:
'''#Testing law selection automation 

#automaticaly detect the best law based on p_value
for each in [ks_test_normal, ks_test_expon, ks_test_uniform]:
    threshold = 0.05
    if each.p_value() > threshold:
        print(f'data is following {each} law')'''

In [ ]:
#Depending on KS results

#norm
noise_norm = np.random.normal(0, 0.5, data.shape)
evolved_data_norm = data + noise

#expon
noise_expon = np.random.exponential(0, 0.5, data.shape)
evolved_data_expon = data + noise

#uniform
noise_uni = np.random.uniform(0, 0.5, data.shape)
evolved_data_uni = data + noise

In [ ]:
# We will first accept as true that data follows a normal law

# Perform clustering on both datasets
n_clusters = kmeans_original.n_clusters

kmeans_original = KMeans(n_clusters=n_clusters, random_state=42).fit(data)
kmeans_evolved = KMeans(n_clusters=n_clusters, random_state=42).fit(evolved_data_norm)

# Calculate ARI
ari_score = adjusted_rand_score(kmeans_original.labels_, kmeans_evolved.labels_)
print(f'Adjusted Rand Index: {ari_score:.3f}')